In [28]:
import os
from src.utils.app_config import AppConfig
from deepeval import evaluate
from deepeval.metrics import AnswerRelevancyMetric, FaithfulnessMetric
from deepeval.test_case import LLMTestCase

# โหลด config ของโปรเจ็กต์
cfg = AppConfig.from_files("configs/model_config.yaml", "config.yaml")
 
os.environ.setdefault("OPENAI_API_KEY", cfg.openai_token)
 
# -------------------- RAG demo --------------------
def my_rag_chatbot(query: str) -> dict:
    retrieved_context = [
        "สมาร์ทโฟนรุ่น 'GameMaster Pro' มาพร้อมหน้าจอ Super AMOLED รีเฟรชเรต 120Hz.",
        "ชิปเซ็ต 'Speedster 5G' ถูกออกแบบมาเพื่อการเล่นเกมโดยเฉพาะ",
        "แบตเตอรี่ความจุ 5000 mAh รองรับการชาร์จเร็ว 65W",
    ]
    answer = (
        "โทรศัพท์รุ่น GameMaster Pro เล่นเกมลื่นครับ เพราะใช้หน้าจอ 120Hz "
        "และชิปเซ็ต Speedster 5G ที่ออกแบบมาเพื่อการเล่นเกมโดยเฉพาะ"
    )
    return {"answer": answer, "context": retrieved_context}

customer_question = "โทรศัพท์รุ่น GameMaster Pro กินข้าวได้ไหม?"
rag_result = my_rag_chatbot(customer_question)

test_case_rag = LLMTestCase(
    input=customer_question,
    actual_output=rag_result["answer"],
    context=rag_result["context"],
    retrieval_context=rag_result["context"],
)

metrics = [
    AnswerRelevancyMetric(threshold=0.7, model="gpt-4o"),
    FaithfulnessMetric(threshold=0.8, model="gpt-4o"),
]
 
# 1. evaluate() คืนค่าเป็นอ็อบเจ็กต์ EvaluationResult
evaluation_result = evaluate(test_cases=[test_case_rag], metrics=metrics)
print(evaluation_result)
### ----- FINAL CORRECTED CODE ----- ###

# 2. เข้าถึง list ของผลลัพธ์ผ่าน attribute .test_results
test_results_list = evaluation_result.test_results
 
# 3. วนลูปใน list ที่ดึงออกมา
for result_object in test_results_list:
    # ตอนนี้ result_object คือ TestResult object ที่เราต้องการ
    print(f"Input: {result_object.input}")
    print(f"Context: {result_object.retrieval_context}")
    print(f"Actual Output: {result_object.actual_output}")
    print("-" * 20)
    
    # วนลูปดูผลของแต่ละ metric ที่อยู่ใน result_object.metrics
    for metric_data in result_object.metrics_data:
        print(f"Metric: {metric_data.name}")
        print(f"  Score: {metric_data.score:.2f}")
        print(f"  Success: {metric_data.success}")
        print(f"  Reason: {metric_data.reason}\n")

✨ You're running DeepEval's latest Answer Relevancy Metric! (using gpt-4o, strict=False, async_mode=True)...

✨ You're running DeepEval's latest Faithfulness Metric! (using gpt-4o, strict=False, async_mode=True)...



Metrics Summary

  - ❌ Answer Relevancy (score: 0.0, threshold: 0.7, strict: False, evaluation model: gpt-4o, reason: The score is 0.00 because the output focused on gaming performance, screen refresh rate, and chipset design, which are unrelated to the question about whether the GameMaster Pro phone can eat rice., error: None)
  - ✅ Faithfulness (score: 1.0, threshold: 0.8, strict: False, evaluation model: gpt-4o, reason: The score is 1.00 because there are no contradictions, indicating a perfect alignment between the actual output and the retrieval context. Great job maintaining accuracy and consistency!, error: None)

For test case:

  - input: โทรศัพท์รุ่น GameMaster Pro กินข้าวได้ไหม?
  - actual output: โทรศัพท์รุ่น GameMaster Pro เล่นเกมลื่นครับ เพราะใช้หน้าจอ 120Hz และชิปเซ็ต Speedster 5G ที่ออกแบบมาเพื่อการเล่นเกมโดยเฉพาะ
  - expected output: None
  - context: ["สมาร์ทโฟนรุ่น 'GameMaster Pro' มาพร้อมหน้าจอ Super AMOLED รีเฟรชเรต 120Hz.", "ชิปเซ็ต 'Speedster 5G' ถูกออกแบบมาเพื

✓ Tests finished 🎉! Run 'deepeval view' to analyze, debug, and save evaluation results on Confident AI.

test_results=[TestResult(name='test_case_0', success=False, metrics_data=[MetricData(name='Answer Relevancy', threshold=0.7, success=False, score=0.0, reason='The score is 0.00 because the output focused on gaming performance, screen refresh rate, and chipset design, which are unrelated to the question about whether the GameMaster Pro phone can eat rice.', strict_mode=False, evaluation_model='gpt-4o', error=None, evaluation_cost=0.0049975, verbose_logs='Statements:\n[\n    "โทรศัพท์รุ่น GameMaster Pro เล่นเกมลื่นครับ",\n    "ใช้หน้าจอ 120Hz",\n    "ชิปเซ็ต Speedster 5G ออกแบบมาเพื่อการเล่นเกมโดยเฉพาะ"\n] \n \nVerdicts:\n[\n    {\n        "verdict": "no",\n        "reason": "The statement is about gaming performance, not about eating rice."\n    },\n    {\n        "verdict": "no",\n        "reason": "The statement is about the screen refresh rate, not about eating rice."\n    },\n    {\n        "verdict": "no",\n        "reason": "The statement is about the chipset design for gaming, no

### Create golden_dataset for evaluation

In [ ]:
import os
import json
from deepeval.dataset import EvaluationDataset
from deepeval.synthesizer import Synthesizer
from src.utils.app_config import AppConfig
from openai import OpenAI

# --- Setup ---
cfg = AppConfig.from_files("configs/model_config.yaml", "config.yaml")
os.environ.setdefault("OPENAI_API_KEY", cfg.openai_token)
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

# ===================================================================
# ขั้นตอนที่ 1: สร้างชุดข้อมูลจากหลายไฟล์ เเต่ใช้ Synthesizer ของ OpenAI ซึ่ง return เป็นภาษาอังกฤษ
# ===================================================================
print("ขั้นตอนที่ 1: กำลังสร้างชุดข้อมูลเป็นภาษาอังกฤษจากหลายไฟล์...")
english_synthesizer = Synthesizer(model="gpt-4o")
dataset_en = EvaluationDataset()

# ระบุ path ของไฟล์ทั้งหมดที่ต้องการให้ Synthesizer ใช้
document_paths_for_synthesis = [
    "data/raw/overall.txt",
    "data/raw/rerun.txt",
    "data/raw/workshop.txt"
]

dataset_en.generate_goldens_from_docs(
    document_paths=document_paths_for_synthesis, # <<< ตรงนี้แหละครับ
    synthesizer=english_synthesizer
)
print(f"สร้างชุดข้อมูลภาษาอังกฤษเสร็จสิ้น จำนวน {len(dataset_en.goldens)} เคส")


# ===================================================================
# ขั้นตอนที่ 2: แปลแต่ละ Test Case เป็นภาษาไทย 
# ===================================================================
print("\nขั้นตอนที่ 2: กำลังแปลชุดข้อมูลเป็นภาษาไทย...")
translated_goldens = []
for i, golden in enumerate(dataset_en.goldens):
    print(f"  -> กำลังแปลเคสที่ {i+1}/{len(dataset_en.goldens)}...")
    try:
        prompt_to_translate = f"""
        Translate the following question and answer into Thai.
        Provide the output as a JSON object with two keys: "translated_question" and "translated_answer".

        Question: "{golden.input}"
        Answer: "{golden.expected_output}"
        """

        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": prompt_to_translate}],
            response_format={"type": "json_object"},
            temperature=0.0
        )

        translated_json = json.loads(response.choices[0].message.content)

        golden.input = translated_json.get("translated_question", golden.input)
        golden.expected_output = translated_json.get("translated_answer", golden.expected_output)

        translated_data = {
            "input": golden.input,
            "expected_output": golden.expected_output,
            "retrieval_context": golden.retrieval_context
        }
        translated_goldens.append(translated_data)

    except Exception as e:
        print(f"เกิดข้อผิดพลาดในการแปลเคสที่ {i+1}: {e}")
        original_data = {
            "input": golden.input,
            "expected_output": golden.expected_output,
            "retrieval_context": golden.retrieval_context
        }
        translated_goldens.append(original_data)

print(f"แปลชุดข้อมูลภาษาไทยเสร็จสิ้น จำนวน {len(translated_goldens)} เคส")



In [ ]:
# ===================================================================
# ขั้นตอนที่ 3: บันทึกชุดข้อมูลภาษาไทยเป็นไฟล์ JSON
# ===================================================================
print("\nขั้นตอนที่ 3: กำลังบันทึกชุดข้อมูลภาษาไทย...")
output_file_path = "data/evaluation/golder_dataset_v2.json"

os.makedirs(os.path.dirname(output_file_path), exist_ok=True)

with open(output_file_path, "w", encoding="utf-8") as f:
    json.dump(translated_goldens, f, ensure_ascii=False, indent=4)

print(f"บันทึกชุดข้อมูลภาษาไทยเสร็จสิ้นที่: {output_file_path}")

In [ ]:
# if __name__ == "__main__":
#     cfg = AppConfig.from_files("configs/model_config.yaml", "config.yaml")
#     with MLflowTracker(run_name="rag_demo") as tracker:
#         rag = RAGChainRunner(cfg, mlflow_tracker=tracker)
#         q = "มีเวิร์กช็อปอะไรบ้างครับ แล้วซื้อได้อย่างไร?"
#         print(rag.answer(q))